In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS databricks_catalog.reporting;


#####View: Daily revenue[total revenue + trips per day.]

In [0]:
%sql
CREATE OR REPLACE VIEW databricks_catalog.reporting.v_daily_revenue AS
SELECT
  trip_date,
  COUNT(*) AS trips,
  SUM(total_amount) AS total_revenue,
  AVG(total_amount) AS avg_trip_revenue
FROM databricks_catalog.gold.fact_trip
GROUP BY trip_date;


#####View:Trips by pickup zone

In [0]:
%sql
CREATE OR REPLACE VIEW databricks_catalog.reporting.v_trips_by_pickup_zone AS
SELECT
  f.trip_date,
  l.borough AS pickup_borough,
  l.zone    AS pickup_zone,
  COUNT(*)  AS trips,
  SUM(f.total_amount) AS total_revenue
FROM databricks_catalog.gold.fact_trip f
LEFT JOIN databricks_catalog.gold.dim_location_scd2 l
  ON f.pu_location_sk = l.location_sk
 AND l.is_current = true
GROUP BY
  f.trip_date, l.borough, l.zone;


#####Historic Trips by Pickup Zone (as-of pickup time)

In [0]:
%sql
CREATE OR REPLACE VIEW databricks_catalog.reporting.v_trips_by_pickup_zone_historic AS
SELECT
  f.trip_date,
  l.borough AS pickup_borough,
  l.zone    AS pickup_zone,
  COUNT(*)  AS trips,
  SUM(f.total_amount) AS total_revenue
FROM databricks_catalog.gold.fact_trip f
LEFT JOIN databricks_catalog.gold.dim_location_scd2 l
  ON f.pu_location_sk = l.location_sk
 AND f.pickup_ts >= l.effective_from
 AND f.pickup_ts <  COALESCE(l.effective_to, TIMESTAMP '2999-12-31 00:00:00')
GROUP BY
  f.trip_date, l.borough, l.zone;


#####Historic Trips by Dropoff Zone (as-of dropoff time)

In [0]:
%sql
CREATE OR REPLACE VIEW databricks_catalog.reporting.v_trips_by_dropoff_zone_historic AS
SELECT
  f.trip_date,
  l.borough AS dropoff_borough,
  l.zone    AS dropoff_zone,
  COUNT(*)  AS trips,
  SUM(f.total_amount) AS total_revenue
FROM databricks_catalog.gold.fact_trip f
LEFT JOIN databricks_catalog.gold.dim_location_scd2 l
  ON f.do_location_sk = l.location_sk
 AND f.dropoff_ts >= l.effective_from
 AND f.dropoff_ts <  COALESCE(l.effective_to, TIMESTAMP '2999-12-31 00:00:00')
GROUP BY
  f.trip_date, l.borough, l.zone;


######Historic Pickup→Dropoff Flow

In [0]:
%sql
CREATE OR REPLACE VIEW databricks_catalog.reporting.v_zone_flow_historic AS
SELECT
  f.trip_date,
  pu.borough AS pickup_borough,
  pu.zone    AS pickup_zone,
  do.borough AS dropoff_borough,
  do.zone    AS dropoff_zone,
  COUNT(*)   AS trips,
  SUM(f.total_amount) AS total_revenue
FROM databricks_catalog.gold.fact_trip f
LEFT JOIN databricks_catalog.gold.dim_location_scd2 pu
  ON f.pu_location_sk = pu.location_sk
 AND f.pickup_ts >= pu.effective_from
 AND f.pickup_ts <  COALESCE(pu.effective_to, TIMESTAMP '2999-12-31 00:00:00')
LEFT JOIN databricks_catalog.gold.dim_location_scd2 do
  ON f.do_location_sk = do.location_sk
 AND f.dropoff_ts >= do.effective_from
 AND f.dropoff_ts <  COALESCE(do.effective_to, TIMESTAMP '2999-12-31 00:00:00')
GROUP BY
  f.trip_date, pu.borough, pu.zone, do.borough, do.zone;


#####View: Payment type summary

In [0]:
%sql
CREATE OR REPLACE VIEW databricks_catalog.reporting.v_payment_type_summary AS
SELECT
  f.trip_date,
  p.payment_type,
  COUNT(*) AS trips,
  SUM(f.total_amount) AS total_revenue,
  AVG(f.tip_amount) AS avg_tip
FROM databricks_catalog.gold.fact_trip f
LEFT JOIN databricks_catalog.gold.dim_payment_type_scd1 p
  ON f.payment_sk = p.payment_sk
GROUP BY
  f.trip_date, p.payment_type;
